<a href="https://colab.research.google.com/github/saoleksii/Trump2020_CSS_course_project/blob/main/Trump2020_CSS_course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import zipfile
import os
import time

Unzip dataset

In [ ]:
#!unzip "/content/drive/MyDrive/Trump2020_CSS_course_project/political_dataset.zip" -d "/content/drive/MyDrive/Trump2020_CSS_course_project/"

File path

In [3]:
file_path = '/content/drive/MyDrive/Trump2020_CSS_course_project/political_dataset.csv'

# Statistics for not prepared dataset

We use chunks to process 6gb file

In [7]:

num_rows = 0
total_submissions = 0
total_comments = 0
frequency_by_subreddit = {}
size = os.path.getsize(file_path) / (1024**2)
print(f'{size} mb')
i = 0
df = pd.read_csv(file_path, chunksize=1000000, low_memory=False)
for idx, chunk in enumerate(df):
    i+=1
    if idx == 0:
        print("\nDataframe overview")
        print(chunk.head(10))
        print("\n")
    num_rows+=len(chunk)
    chunk[['title', 'body']] = chunk[['title', 'body']].fillna('')
    submissions_in_chunk = chunk[chunk['title'] != ''].shape[0]
    comments_in_chunk = chunk[chunk['body'] != ''].shape[0]
    total_submissions += submissions_in_chunk
    total_comments += comments_in_chunk
    current_frequency = chunk['subreddit'].value_counts().to_dict()
    for sub, count in current_frequency.items():
        frequency_by_subreddit[sub] = frequency_by_subreddit.get(sub, 0) + count
    print(f'{i} of 38 chunks processed')

print("Completed\n")

metrics_data = {
    'Total Rows': f"{num_rows}",
    'Total Size (MB)': f"{size}",
    'Submissions Count': f"{total_submissions}",
    'Comments Count': f"{total_comments}",
}
print("Summary of Basic Dataset Metrics:")
print(pd.Series(metrics_data))
print("\nSubreddit_frequency:")
print(pd.Series(frequency_by_subreddit).sort_values(ascending = False))

6886.546831130981 mb

Dataframe overview
      subreddit   created_utc  title  selftext  \
0  Conservative  1.609459e+09    NaN       NaN   
1  Conservative  1.609459e+09    NaN       NaN   
2  Conservative  1.609459e+09    NaN       NaN   
3  Conservative  1.609459e+09    NaN       NaN   
4  Conservative  1.609459e+09    NaN       NaN   
5  Conservative  1.609459e+09    NaN       NaN   
6  Conservative  1.609459e+09    NaN       NaN   
7  Conservative  1.609459e+09    NaN       NaN   
8  Conservative  1.609459e+09    NaN       NaN   
9  Conservative  1.609459e+09    NaN       NaN   

                                                body  
0                           The Boer will rise again  
1  Tired of reporting this thread? Debate us on d...  
2                                          [removed]  
3                                          [removed]  
4                                          [removed]  
5  Lol.  U wish that much was going to stimulus c...  
6                      

# Preparing dataset:

Check the data by looking at one chunk

In [9]:
df = pd.read_csv(file_path, chunksize=500000)
chunk = next(df) #first chunk
print(chunk.head(10))
chunk[['title', 'selftext', 'body']] = chunk[['title', 'selftext', 'body']].fillna('') # замінити всі NaN на пустий рядок
print(chunk[['title', 'selftext', 'body']].value_counts().head(50))

      subreddit   created_utc  title  selftext  \
0  Conservative  1.609459e+09    NaN       NaN   
1  Conservative  1.609459e+09    NaN       NaN   
2  Conservative  1.609459e+09    NaN       NaN   
3  Conservative  1.609459e+09    NaN       NaN   
4  Conservative  1.609459e+09    NaN       NaN   
5  Conservative  1.609459e+09    NaN       NaN   
6  Conservative  1.609459e+09    NaN       NaN   
7  Conservative  1.609459e+09    NaN       NaN   
8  Conservative  1.609459e+09    NaN       NaN   
9  Conservative  1.609459e+09    NaN       NaN   

                                                body  
0                           The Boer will rise again  
1  Tired of reporting this thread? Debate us on d...  
2                                          [removed]  
3                                          [removed]  
4                                          [removed]  
5  Lol.  U wish that much was going to stimulus c...  
6                                          [removed]  
7        

Delete top 50 frequent comments

In [10]:
chunk['created_utc'] = pd.to_datetime(chunk['created_utc'], unit='s') #edit time
#delete useless comments
top_50_noise = chunk['body'].value_counts().head(50).index.tolist()
rows_to_drop = chunk['body'].isin(top_50_noise)
indices_to_drop = chunk.index[rows_to_drop]
chunk.drop(indices_to_drop, inplace=True)
print(chunk['body'].value_counts().head(5))

body
Amen!         20
Thank you.    20
Ok            20
Nah           19
Thanks!       19
Name: count, dtype: int64


Check the size of chunk after removing useless comments

In [11]:
print(chunk.shape)

(298777, 5)


We reduced the number of rows from 500000 to 300000.
Now make one column for title, selftext and body

In [12]:
chunk['text'] = (chunk['title'] + ' ' + chunk['selftext'] + ' ' + chunk['body'])
chunk['text'] = chunk['text'].str.lower().str.strip()
chunk.drop(columns=['title', 'selftext', 'body'], inplace=True)
print(chunk.sample(10))

           subreddit         created_utc  \
351190  Conservative 2021-01-21 00:11:06   
233012  Conservative 2021-01-14 18:04:05   
474639  Conservative 2021-01-25 18:55:22   
420222  Conservative 2021-01-23 01:15:39   
446949  Conservative 2021-01-24 06:36:59   
17530   Conservative 2021-01-03 17:53:09   
385996  Conservative 2021-01-21 22:18:10   
184701  Conservative 2021-01-12 06:04:54   
328349  Conservative 2021-01-20 17:19:39   
259660  Conservative 2021-01-16 22:42:54   

                                                     text  
351190                                   what’s in march?  
233012  against kamala as a challenger? if our electio...  
474639  interesting that he’s now saying what has been...  
420222  this was not where they were lodging/sleeping....  
446949  well, really the northern states wanted them t...  
17530   this particular individual does not have a tatoo.  
385996  when gas hits $8 gallon, truckers will strike ...  
184701                             

If text is less than 7 symbols it is useless so we delete everything that is less than it

In [13]:
drop_rows = chunk['text'].str.len() <= 7
indices_to_drop = chunk.index[drop_rows]
chunk.drop(indices_to_drop, inplace = True)
print(chunk.sample(10))
print(chunk.shape)

           subreddit         created_utc  \
129195  Conservative 2021-01-09 03:49:45   
250367  Conservative 2021-01-16 00:09:02   
120752  Conservative 2021-01-08 23:28:14   
24104   Conservative 2021-01-04 03:11:53   
454539  Conservative 2021-01-24 18:19:34   
141748  Conservative 2021-01-09 21:26:48   
39558   Conservative 2021-01-06 00:33:11   
371189  Conservative 2021-01-21 15:09:04   
353623  Conservative 2021-01-21 01:16:55   
210257  Conservative 2021-01-13 19:34:06   

                                                     text  
129195                                        3-4 months.  
250367  right, because you're the pinnacle of individu...  
120752  they make the rules, so of course they are goi...  
24104   a nice chipotle mayo helps the crow go down......  
454539  i'm just not in to arguing with internet stran...  
141748  there’s an article on bloomberg saying some ho...  
39558                      there should be special flair!  
371189  umm. perhaps you should che

Now select key words to keep only rows that are needed

In [15]:
keyword_list = [
    'trump', 'biden', 'pence', 'capitol', 'capitol hill', 'impeach', 'impeachment',
    'january 6', '6th jan', 'election', 'stolen', 'steal', 'fraud', 'stop the steal',
    'deep state', 'conspiracy', 'patriot', 'traitors', 'treason', 'rigged',
    'censor', 'ban', 'removed', 'suspended', 'free speech', 'freedom of speech',
    'twitter ban', 'parler', 'maga'
]
keywords = '|'.join(keyword_list)
rows_to_drop = ~chunk['text'].str.contains(keywords)
indices_to_drop = chunk.index[rows_to_drop]
new_chunk = chunk.drop(indices_to_drop)
print(new_chunk.head(10))
print(new_chunk.shape)

       subreddit         created_utc  \
21  Conservative 2021-01-01 00:03:03   
23  Conservative 2021-01-01 00:03:34   
32  Conservative 2021-01-01 00:04:34   
46  Conservative 2021-01-01 00:07:13   
51  Conservative 2021-01-01 00:08:50   
56  Conservative 2021-01-01 00:09:26   
61  Conservative 2021-01-01 00:10:29   
76  Conservative 2021-01-01 00:12:40   
84  Conservative 2021-01-01 00:14:09   
94  Conservative 2021-01-01 00:15:37   

                                                 text  
21    i wonder how trump feels about pence right now?  
23  this man got his upvotes silenced and then got...  
32  of course this is what the demoncrats want. a ...  
46  a bank is a private business and should be abl...  
51                     or tried for treason/sedition?  
56  let's be real, the optics of pence literally p...  
61  i have a feeling trump knows. both trump and p...  
76  &gt;\ti honestly think he’s purposely trying t...  
84  when has trump supporters rioted or caused vio...  

Apply for all chunks

In [17]:
output_file = '/content/drive/MyDrive/Trump2020_CSS_course_project/processed_dataset.csv'
num_rows = 0
frequency_by_subreddit = {}
i = 0
df = pd.read_csv(file_path, chunksize=1000000, low_memory=False)
for idx, chunk in enumerate(df):
  chunk[['title', 'selftext', 'body']] = chunk[['title', 'selftext', 'body']].fillna('') # replace NaN with empty line
  chunk.rename(columns={'created_utc': 'time_created'}, inplace=True)
  chunk['time_created'] = pd.to_datetime(chunk['time_created'], unit='s') #edit time
  #delete useless comments
  top_50_noise = chunk['body'].value_counts().head(50).index.tolist()
  rows_to_drop = chunk['body'].isin(top_50_noise)
  indices_to_drop = chunk.index[rows_to_drop]
  chunk.drop(indices_to_drop, inplace=True)
  #обʼєднати колонки текстові
  chunk['text'] = (chunk['title'] + ' ' + chunk['selftext'] + ' ' + chunk['body'])
  chunk['text'] = chunk['text'].str.lower().str.strip()
  chunk.drop(columns=['title', 'selftext', 'body'], inplace=True)
  #delete less than 7 symbols
  drop_rows = chunk['text'].str.len() <= 7
  indices_to_drop = chunk.index[drop_rows]
  chunk.drop(indices_to_drop, inplace = True)
  #select keywords
  keyword_list = [
    'trump', 'biden', 'pence', 'capitol', 'capitol hill', 'impeach', 'impeachment',
    'january 6', '6th jan', 'election', 'stolen', 'steal', 'fraud', 'stop the steal',
    'deep state', 'conspiracy', 'patriot', 'traitors', 'treason', 'rigged',
    'censor', 'ban', 'removed', 'suspended', 'free speech', 'freedom of speech',
    'twitter ban', 'parler', 'maga'
    ]
  keywords = '|'.join(keyword_list)
  rows_to_drop = ~chunk['text'].str.contains(keywords)
  indices_to_drop = chunk.index[rows_to_drop]
  chunk.drop(indices_to_drop, inplace=True)

  #статистика
  i+=1
  if idx == 0:
    print("\nDataframe overview")
    print(chunk.head(10))
    print(chunk.shape)
    print("\n")
  num_rows+=len(chunk)
  current_frequency = chunk['subreddit'].value_counts().to_dict()
  for sub, count in current_frequency.items():
      frequency_by_subreddit[sub] = frequency_by_subreddit.get(sub, 0) + count

  #processed csv file
  if idx == 0:
      header_flag = True
      mode_flag = 'w'
  else:
      header_flag = False
      mode_flag = 'a'
  chunk.to_csv(output_file, index=False, mode=mode_flag, header=header_flag, encoding='utf-8')
  print(f'{i} of 38 chunks processed')

print("Completed\n")

size = os.path.getsize(output_file) / (1024**2)

metrics_data = {
    'Total Rows': f"{num_rows}",
    'Total Size (MB)': f"{size}",
}
print("Summary of Basic Dataset Metrics:")
print(pd.Series(metrics_data))
print("\nSubreddit_frequency:")
print(pd.Series(frequency_by_subreddit).sort_values(ascending = False))

6886.546831130981 mb

Dataframe overview
       subreddit        time_created  \
21  Conservative 2021-01-01 00:03:03   
23  Conservative 2021-01-01 00:03:34   
32  Conservative 2021-01-01 00:04:34   
46  Conservative 2021-01-01 00:07:13   
51  Conservative 2021-01-01 00:08:50   
56  Conservative 2021-01-01 00:09:26   
61  Conservative 2021-01-01 00:10:29   
76  Conservative 2021-01-01 00:12:40   
84  Conservative 2021-01-01 00:14:09   
94  Conservative 2021-01-01 00:15:37   

                                                 text  
21    i wonder how trump feels about pence right now?  
23  this man got his upvotes silenced and then got...  
32  of course this is what the demoncrats want. a ...  
46  a bank is a private business and should be abl...  
51                     or tried for treason/sedition?  
56  let's be real, the optics of pence literally p...  
61  i have a feeling trump knows. both trump and p...  
76  &gt;\ti honestly think he’s purposely trying t...  
84  when has t